In [ ]:
!pip install torchviz

In [ ]:
!pip install qiskit
!pip install qiskit-aer

In [ ]:
!pip install qiskit_machine_learning

In [ ]:
!pip install torchmetrics

In [ ]:
!pip install qiskit-algorithms

In [ ]:
import numpy as np
import torch
from torchvision.transforms import ToTensor
from torchvision.models import vgg16, VGG16_Weights
from torch import cat, no_grad, manual_seed
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (Module, Conv2d, Linear, Dropout2d, CrossEntropyLoss, MaxPool2d, Flatten, Sequential, ReLU)
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
from torchviz import make_dot
import torchvision
from torchvision.io import read_image
from torch.autograd import Variable
import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *
from qiskit import QuantumCircuit
import qiskit_aer
import qiskit.primitives
import qiskit.compiler
import qiskit.quantum_info
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from qiskit.primitives import Estimator
from qiskit.quantum_info import Pauli
import torchmetrics
import qiskit_algorithms

In [ ]:
feature_map = ZZFeatureMap(feature_dimension=2, entanglement='linear')
ansatz = RealAmplitudes(2, reps=1, entanglement='linear')
circuit = QuantumCircuit(2)
circuit.compose(feature_map, range(2), inplace=True)
circuit.compose(ansatz, range(2), inplace=True)
qnn2 = EstimatorQNN(circuit=circuit.decompose(), input_gradients=True, input_params=feature_map.parameters,
    weight_params=ansatz.parameters,)
print(qnn2)

In [ ]:
class Net(Module):

    def __init__(self):
        super().__init__()
        # Load pre-trained VGG16 model
        pretrained_model = vgg16(weights=VGG16_Weights.DEFAULT)

        # Freeze the pre-trained model
        for param in pretrained_model.parameters():
            param.requires_grad = False

        # Define the feature extractor
        self.feature_extractor = nn.Sequential(*list(pretrained_model.children())[:-2])

        # Replace first two convolutional layers with extracted features
        self.conv1 = self.feature_extractor
        self.conv2 = Conv2d(1280, 32, kernel_size=5)

        # Remaining convolutional layers
        self.conv3 = Conv2d(32, 64, kernel_size=2)
        self.conv4 = Conv2d(64, 128, kernel_size=1)
        self.conv5 = Conv2d(128, 256, kernel_size=1)

        # Dropout layer
        self.dropout1 = Dropout2d(0.5)
        self.dropout2 = Dropout2d(0.5)

        # Fully connected layers
        self.fc1 = Linear(256, 64)
        self.fc2 = Linear(64, 2)

        # QNN
        self.qnn = TorchConnector(qnn2)

        # Final layer
        self.fc3 = Linear(1, 1)

    def forward(self, x):
        features = self.conv1(x)
        x = F.relu(self.conv2(features))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = self.dropout1(x)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.dropout2(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.qnn(x)  # apply QNN
        x = self.fc3(x)
        return torch.cat((x, 1 - x), -1)

In [ ]:
dataset = torchvision.datasets.ImageFolder(root="/kaggle/input/sarscov2-ctscan-dataset/")
print(dataset)

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ["non-COVID", "COVID"]
        self.image_folder = torchvision.datasets.ImageFolder(root=self.root_dir, transform=self.transform)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img, label = self.image_folder[idx]
        adjusted_label = 0 if label == 0 else 1
        return img, adjusted_label

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

# Create the custom dataset
dataset = CustomDataset(root_dir="/kaggle/input/sarscov2-ctscan-dataset/", transform=transform)

In [ ]:
print(dataset.__len__())

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model = Net()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

In [ ]:
model.to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import matplotlib.pyplot as plt

# ... (model, device, train_loader, test_loader definitions) ...

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(device)
confusion_matrix = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=2).to(device)


epochs = 50
train_loss_list = []
train_accuracy_list = []
val_loss_list = []
val_accuracy_list = []

for epoch in range(epochs):
    model.train()  # Set model to training mode

    train_total_loss = []
    accuracy.reset()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        # Forward pass
        output = model(data)

        # Calculate training loss
        loss = loss_func(output, target)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_total_loss.append(loss.item())
        accuracy.update(output, target)

    # Compute and store training epoch metrics
    epoch_train_accuracy = accuracy.compute().item()
    epoch_train_loss = sum(train_total_loss) / len(train_total_loss)

    train_accuracy_list.append(epoch_train_accuracy)
    train_loss_list.append(epoch_train_loss)

    # Validation phase
    model.eval()  # Set model to evaluation mode

    val_total_loss = []
    accuracy.reset()
    confusion_matrix.reset()

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            # Forward pass
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True).squeeze()
            # Calculate validation loss
            loss = loss_func(output, target)

            val_total_loss.append(loss.item())
            accuracy.update(output, target)# Assuming pred already calculated
            confusion_matrix.update(pred, target)

    # Compute and store validation epoch metrics
    epoch_val_accuracy = accuracy.compute().item()
    epoch_val_loss = sum(val_total_loss) / len(val_total_loss)

    val_accuracy_list.append(epoch_val_accuracy)
    val_loss_list.append(epoch_val_loss)

    # Print epoch results
    print(f'Epoch [{epoch + 1}/{epochs}]\tTrain Loss: {epoch_train_loss:.4f}\tTrain Accuracy: {epoch_train_accuracy:.4f}\tVal Loss: {epoch_val_loss:.4f}\tVal Accuracy: {epoch_val_accuracy:.4f}')


In [ ]:
# Plot training and validation metrics
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracy_list, label='Train Accuracy')
plt.plot(val_accuracy_list, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy per Epoch')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_loss_list, label='Train Loss')
plt.plot(val_loss_list, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss per Epoch')
plt.grid(True)
plt.legend()

plt.show()

cm = confusion_matrix.compute()
print("Confusion Matrix:\n", cm)
fig_, ax_ = confusion_matrix.plot()

In [ ]:

n_samples_show = 5
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(15, 5))

model.eval()
with no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        data = data.to(device)
        target = target.to(device)
        if count == n_samples_show:
            break
        output = model(data[0:1])
        if len(output.shape) == 1:
            output = output.reshape(1, *output.shape)

        pred = output.argmax(dim=1, keepdim=True)
        axes[count].imshow(data[0].cpu().numpy().squeeze(), cmap='gray')

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        if pred.item() == 0:
            axes[count].set_title('Predicted item: non-COVID')  
        elif pred.item() == 1:
            axes[count].set_title('Predicted item: COVID')  
        count += 1